In [ ]:
import numpy as np
import pandas as pd
import os
import pickle
from bottleneck import push
from downstream_eval import *

load datasets

In [ ]:
GP_VAE=np.load(".....npy")
E2EGAN=np.load(".....npy")

In [ ]:
exp_name = "....IGNITE"
d = np.load('data/'+exp_name+'.npz')

In [ ]:
def renormalize(data, means, stds):
    renorms=[]
    for patient in data:
        r= patient*stds+ means
        renorms.append(r)
    renorm_full = np.stack(renorms, axis=0)
    return(renorm_full)
def normalize(data, means, stds):
    renorms=[]
    for patient in data:
        r= (patient-means) /stds
        renorms.append(r)
    renorm_full = np.stack(renorms, axis=0)
    return(renorm_full)

In [ ]:
data_path = 'extracts/'
with open(os.path.join(data_path, 'mask_combined.pkl'), 'rb') as f:
    miss = pickle.load(f)
with open(os.path.join(data_path, 'LOCV_combined.pkl'), 'rb') as f:
    LOCF = pickle.load(f)
with open(os.path.join(data_path, 'normalized_combined.pkl'), 'rb') as f:
    original = pickle.load(f)
with open(os.path.join(data_path, 'out_combined.pkl'), 'rb') as f:
    outcomes = pickle.load(f)
zero=np.where(np.isnan(original), 0, original)

Get Mean & STD

In [ ]:
means = []
stds = []
flatten= original.reshape(original.shape[0]*original.shape[1], original.shape[2])
for i in range(original.shape[2]):
    means.append(np.nanmean(flatten[:,i]))
    stds.append(np.nanstd(flatten[:,i]))

In [ ]:
array= np.array(miss, dtype=float)
imputed_ours =(array * (original)+ ((1-array) *d["IMM_rec"]))
#imputed_GP =(array * (original[:,:,:35])+ ((1-array) *GP_VAE[:,:,:35]))

## General performance evaluation

In [ ]:
get_results(["Mean","LOCF","IGNITE"],[zero,LOCF,imputed_ours],outcomes)

## Sample-wise missingness results


In [ ]:
# choose the min and max allowed sample-wise missingness (0%, 80%)
get_result_sample_missingess(["Mean","LOCF","IGNITE"],[zero,LOCF,imputed_ours],outcomes, miss, 0, 0.8)

In [ ]:
# choose the min and max allowed sample-wise missingness (80%, 85%)

get_result_sample_missingess(["Mean","LOCF","IGNITE"],[zero,LOCF,imputed_ours],outcomes, miss, 0.8, 0.85)

In [ ]:
# choose the min and max allowed sample-wise missingness (85%, 100%)

get_result_sample_missingess(["Mean","LOCF","IGNITE"],[zero,LOCF,imputed_ours],outcomes, miss, 0.85, 1.0001)

## Feature-wise missingness results

In [ ]:
# choose the min and max allowed feature-wise missingness (0%, 15%)

get_result_feature_missingess(["Mean","LOCF","IGNITE"],[zero,LOCF,imputed_ours],outcomes, miss, 0.0, 0.15)

In [ ]:
# choose the min and max allowed feature-wise missingness (15%, 45%)

get_result_feature_missingess(["Mean","LOCF","IGNITE"],[zero,LOCF,imputed_ours],outcomes, miss, 0.15, 0.45)

In [ ]:
# choose the min and max allowed feature-wise missingness (45%, 100%)

get_result_feature_missingess(["Mean","LOCF","IGNITE"],[zero,LOCF,imputed_ours],outcomes, miss, 0.45, 1)